In [ ]:
import sys

sys.path.append("../")
from helper.utils import *
from helper.cross_validation import *
from config import *
from helper.categorical_features import * 
from sklearn.cluster import KMeans
import geopandas as gpd
# %load_ext lab_black

In [ ]:
! pwd

### Load data

In [ ]:
train = pd.read_csv(join(raw_data_path,"Train.csv"))
test = pd.read_csv(join(raw_data_path, "Test.csv"))

In [ ]:
other_columns=["NL","lon","lat","total_households","total_individuals"]

In [ ]:
columns_to_remove=["dw_12","dw_13","lan_13","pw_07","pw_08"]
train.drop(columns_to_remove,1,inplace=True)
test.drop(columns_to_remove,1,inplace=True)


In [ ]:
# columns_ratio=list(set([col.split("_")[0] for col in test.columns if col not in other_columns+["ward"] ]))
# for prefix in columns_ratio : 
#     col_opp=[col for col in test.columns if col.startswith(prefix)]
#     train[prefix+"_mean"]=train[col_opp].mean(axis=1)
#     train[prefix+"_std"]=train[col_opp].std(axis=1)
    
# #     train[prefix+"_min"]=train[col_opp].min(axis=1)    
# #     train[prefix+"_max"]=train[col_opp].max(axis=1)    
# #     train[prefix+"_median"]=train[col_opp].median(axis=1)  
    
    
#     test[prefix+"_mean"]=test[col_opp].mean(axis=1)
#     test[prefix+"_std"]=test[col_opp].std(axis=1)
    
# #     test[prefix+"_min"]=test[col_opp].min(axis=1)    
# #     test[prefix+"_max"]=test[col_opp].max(axis=1)    
# #     test[prefix+"_median"]=test[col_opp].median(axis=1)  


In [ ]:
train["train"]=1
test["train"]=0
data=pd.concat([train,test])
lat_lon=data[["lat","lon"]]
data.shape

In [ ]:
cols_to_drop=['dw_00','dw_02', 'dw_06','psa_02','lan_02','lan_03','lan_04',
    'lan_05','lan_08', 'pw_01' , 'lan_07','NL',]
data.drop(cols_to_drop,1,inplace=True)

In [ ]:
data["ADM4_PCODE"]=data["ADM4_PCODE"].apply(lambda x:int(x.replace("ZA","")))
# data["ward_num"]=data.ward.apply(lambda x:x.split(":")[0])
# data["ADM4_PCODE_part_1"]=data["ADM4_PCODE"].apply(lambda x:int( str(x)[0:5]))
# data["ADM4_PCODE_part_2"]=data["ADM4_PCODE"].apply(lambda x: int(str(x)[-3:]))
# data["ADM4_PCODE_part_1_1"]=data["ADM4_PCODE_part_1"].apply(lambda x: int(str(x)[0:3]))
# data["ADM4_PCODE_part_1_2"]=data["ADM4_PCODE_part_1"].apply(lambda x: int(str(x)[-3:]))
# for i in range(5) :
#     data["ADM4_PCODE_part_1"+str(i)]=data["ADM4_PCODE_part_1"].apply(lambda x:int(str(x)[i]))

In [ ]:
# # sacled features 
# from sklearn.preprocessing import MinMaxScaler ,QuantileTransformer
# columns=test.drop(["ward","train"],1).columns
# data[columns]=MinMaxScaler(output_distribution="normal").fit_transform(data[columns])

In [ ]:
# km=KMeans(5,random_state=2019)
# data["lat_lon_cluster"]=km.fit_predict(lat_lon)

In [ ]:
# km=KMeans(10,random_state=2019)
# columns_ratio=list(set([col.split("_")[0] for col in test.columns if col not in other_columns+["ward","train"] ]))
# last_columns=[]
# for prefix in columns_ratio :
#     col_opp=[col for col in test.columns if col.startswith(prefix)]
#     data["km_"+prefix]=km.fit_predict(data[col_opp])
# #     data["km_"+prefix]=km.fit_predict(data[last_columns])
    
#     last_columns.append(col_opp)
    

In [ ]:
# data["ratio_total_households"]=data.total_households/data.total_households.sum()
# data["ratio_total_individuals"]=data.total_individuals/data.total_individuals.sum()
# data["total_households/total_individuals"]=data.total_households/data.total_individuals
# data["city"]=data["ward"].apply(lambda x: str(x)[0:3])


In [ ]:
data['rich'] = data['car_01']+data['stv_00']+data['psa_01']
data['poor'] = data['car_00'] +data['stv_01']+data['psa_00']

In [ ]:
# data["sum_columns"]=data[columns].sum(1)
# for column in  columns :
#     data[column]=data[column].round(2)
#     .map(data[column].round(3).value_counts())

In [ ]:
columns=data.drop(["ward","target","train","ADM4_PCODE"],1).columns

data_km=data[columns].copy()
data_km["total_households"]/=data_km["total_households"].max()
data_km["total_individuals"]/=data_km["total_individuals"].max()

km=KMeans(10,random_state=2019)
data["cluster"]=km.fit_predict(data_km[columns])



In [ ]:
ward_area=pd.read_csv(join(proc_data_path,"ward_area.csv"))
ward_area["ADM4_PCODE"]=ward_area["ADM4_PCODE"].apply(lambda x:int(x.replace("ZA","")))
data=data.merge(ward_area,how="left",on=["ADM4_PCODE"])

In [ ]:
train=data[data.train==1].reset_index(drop=True)
test=data[data.train==0].reset_index(drop=True)

### Cross validation

In [ ]:
CV = cross_validation(
    train_df=train,
    _id_="ward",
    target_name="target",
    kfold_type="kfold",
    output_dir=proc_data_path,
    split_ratio=0.1,
    nfolds=10,
    random_state=1994,
    shuffle=True,
    stratify=False,
)
train = CV.split()

In [ ]:
version = "0"
train.to_csv(join(proc_data_path, "train_{}.csv".format(version)), index=False)
test.to_csv(join(proc_data_path, "test_{}.csv".format(version)), index=False)

In [ ]:
train.fold.unique()